In [56]:
import pandas as pd

In [57]:
import os

path ="../raw_data/"
#we shall store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))

In [58]:
filelist

['../raw_data/raw_10+_1.csv',
 '../raw_data/raw_10.csv',
 '../raw_data/raw_11.csv',
 '../raw_data/raw_12.csv',
 '../raw_data/raw_13.csv',
 '../raw_data/raw_14.csv',
 '../raw_data/raw_2.csv',
 '../raw_data/raw_3.csv',
 '../raw_data/raw_4.csv',
 '../raw_data/raw_5.csv',
 '../raw_data/raw_6.csv',
 '../raw_data/raw_7.csv',
 '../raw_data/raw_8.csv',
 '../raw_data/raw_9.csv']

In [59]:
df=None
for path in filelist:
    _df=pd.read_csv(path)
    df=pd.concat([df,_df])
    print(df.shape[0])

88197
138197
188197
238197
288197
308197
318197
368118
418118
468118
498118
578118
598118
648118


In [60]:
df.shape

(648118, 2)

In [61]:
df.head()

,USDOT,content
0,883,NORTHWEST PIPE FITTINGS INC (USDOT 883) US DOT...
1,1596,FOLTZ LITTER INC (USDOT 1596) US DOT 411 Searc...
2,2340,KORTH TRUCKING INC (USDOT 2340) US DOT 411 Sea...
3,2824,AMERICAN BLOCK COMPANY INC (USDOT 2824) US DOT...
4,4400,CAMPBELL TRACTOR & IMPLEMENT INC (USDOT 4400) ...


In [62]:
df.duplicated().sum()

281479

In [63]:
df = df.drop_duplicates(keep='first')

In [64]:
df.duplicated().sum()

0

In [65]:
df.shape

(366639, 2)

In [66]:
# df.to_csv()

In [67]:
company_info=["Operating status","DBA","Entity type","Operations","Classification","Docket #","DUNS #","SCAC","MCS-150 form date","MCS-150 Mileage/Year","Fleet Details"]
contact_info=["Business address","Phone","Fax","Email","Web","Mailing address","Representative","Commodities"]
fleet_info=["Power units", "Autonomous units", "Electric units", "Drivers", "SmartWay", "CARB TRU","Contacts"]
all_records=[]
allrecords_no_data=[]

In [68]:
def extract(number,cleantext):
    data={}
    #usdot number
    data["USDOT"]=number
    #Company Name
    try:
        data["Company Name"]=cleantext.split("USDOT")[0].replace("(","").strip()
    except:
        data["Company Name"]=None

    company_content=cleantext.split("Company")[1]
    contact_content=cleantext.split("Contacts")[1].split("Building Materials")[0]

    #company details
    for i in range(len(company_info)):
        if company_info[i]=="Fleet Details":
            break
        try:
            data[company_info[i]]=company_content.split(company_info[i+1])[0].split(company_info[i])[1].strip()
        except:
            data[company_info[i]]=None
    
    #contact details
    for i in range(len(contact_info)):
        if contact_info[i]=="Commodities":
            break
        try:
            data[contact_info[i]]=contact_content.split(contact_info[i+1])[0].split(contact_info[i])[1].strip()
        except:
            data[contact_info[i]]=None
    
    #fleet details
    for i in range(len(fleet_info)):
        if fleet_info[i]=="Contacts":
            break
        try:
            data[fleet_info[i]]=company_content.split(fleet_info[i+1])[0].split(fleet_info[i])[1].strip()
        except:
            data[fleet_info[i]]=None
    return data

In [69]:
data,no_data=0,0
for index, row in df.iterrows():
    try:
        output=extract(row["USDOT"],row["content"])
        all_records.append(output)
        data+=1
    except:
        allrecords_no_data.append({"USDOT":row["USDOT"],"content":row["content"]})
        no_data+=1

In [70]:
print(f"Companies with Data    : {data}")
print(f"Companies without Data : {no_data}")

Companies with Data    : 252483
Companies without Data : 114156


In [71]:
data+no_data

366639

In [72]:
df.shape

(366639, 2)

In [73]:
df_with_data = pd.DataFrame.from_dict(all_records)
df_with_data.head()

,USDOT,Company Name,Operating status,DBA,Entity type,Operations,Classification,Docket #,DUNS #,SCAC,...,Email,Web,Mailing address,Representative,Power units,Autonomous units,Electric units,Drivers,SmartWay,CARB TRU
0,883,NORTHWEST PIPE FITTINGS INC,ACTIVE,,Carrier,Interstate,Private Property,,7499718,,...,lblack@northwestpipe.com,www.northwestpipe.com,"P O BOX 920 RAPID CITY, SD 57709-0920",SCOTT BARBOUR DOUG JOHNSON,11,,,7,Not a,Not 100% compliant
1,1596,FOLTZ LITTER INC,ACTIVE,,Carrier,Interstate,Private Property,,,,...,lfoltz@hardynet.com,,"163 MIDDLE COVE ROAD MATHIAS, WV 26812",KEVIN L FOLTZ LORA J FOLTZ,11,,,8,Not a,Not 100% compliant
2,2340,KORTH TRUCKING INC,AUTHORIZED FOR PROPERTY,,Carrier,Interstate,Authorized For Hire Private Property,MC177051,91997437,,...,brad@kcgrain.com,,"55403 HIGHWAY 13 NORFOLK, NE 68701",BRAD KORTH,11,,,11,Not a,Not 100% compliant
3,2824,AMERICAN BLOCK COMPANY INC,ACTIVE,,Carrier,Interstate,Private Property,,5002738,,...,easternvault@frontiernet.net,,"494 COURTHOUSE ROAD PRINCETON, WV 24740",BRIAN STRUBLE BRIAN P. STRUBLE,11,,,7,Not a,Not 100% compliant
4,4400,CAMPBELL TRACTOR & IMPLEMENT INC,ACTIVE,CAMPBELL TRACTOR,Carrier,Interstate,Private Property,,33982703,,...,lbasley@campbelltractor.com,www.campbelltractor.com,"2014 N FRANKLIN BLVD NAMPA, ID 83687",DAN W CAMPBELL,11,,,9,Not a,Not 100% compliant


In [74]:
df_without_data = pd.DataFrame.from_dict(allrecords_no_data)
df_without_data.head()

,USDOT,content
0,100023,USDOT 100023 US DOT 411 Search Monitoring Anal...
1,100044,USDOT 100044 US DOT 411 Search Monitoring Anal...
2,100079,USDOT 100079 US DOT 411 Search Monitoring Anal...
3,100087,USDOT 100087 US DOT 411 Search Monitoring Anal...
4,100097,USDOT 100097 US DOT 411 Search Monitoring Anal...


In [75]:
df_with_data.to_csv (r'../extracted_data/extracted_companies.csv', index = False, header=True)
df_without_data.to_csv (r'../extracted_data/extracted_companies_no_data.csv', index = False, header=True)